In [1]:
import mod_sql as m_s
import pandas as pd

In [17]:
## 클래스 생성
db = m_s.Database()

In [ ]:
## 단순 행 결합
## python에서는 concat()
## sql union 결합
sql = """
        select * from transaction_1 
        union 
        select * from transaction_2
"""
db.executeall(sql)

### 굉장히 단순하지만 노가다성이 뛰어난 sql
1. transaction_1, transaction_2를 union 결합
2. transaction_detail_1, transaction_detail_2를 union 결합
3. 1번 2번 과정에서 나온 테이블을 left join
4. 이 모든 과정이 sql문 한번에 처리 


hint (  select * from 1번 테이블 left join 2번 테이블 on 조건  )

In [ ]:
# 2번 과정 
sql = """
        select * from transaction_detail_1 
        union 
        select * from transaction_detail_2
"""
db.executeall(sql)

In [ ]:
sql = """
    select 
    * 
    from 
    (select * from transaction_1 
    union 
    select * from transaction_2) as tr 
    left join 
    (select * from transaction_detail_1 
    union
    select * from transaction_detail_2) as tr_d 
    on 
    tr.transaction_id = tr_d.transaction_id
"""
db.executeall(sql)

### pandas에서 작업
1. 4개의 테이블을 각각 변수에 담아서 시작
2. concat()과 merge() 함수를 이용하여 sql의 결과와 같은 출력

In [7]:
sql = "select * from transaction_1"
sql2 = "select * from transaction_2"
sql3 = "select * from transaction_detail_1"
sql4 = "select * from transaction_detail_2"

tr1 = db.executeall(sql)
tr2 = db.executeall(sql2)
tr_d1 = db.executeall(sql3)
tr_d2 = db.executeall(sql4)

In [10]:
tr = pd.concat([tr1, tr2], ignore_index=True)
tr_d = pd.concat([tr_d1, tr_d2], ignore_index=True)

In [ ]:
df = pd.merge(tr, tr_d, on='transaction_id', how='left')
df

In [12]:
tr1 = pd.read_csv("../csv/transaction_1.csv")
tr2 = pd.read_csv("../csv/transaction_2.csv")

In [ ]:
pd.concat([tr1, tr2], ignore_index=True)

In [ ]:
## left join transaction_detail_1, item_master 결합
## quantity * item_price 결과값을 price 컬럼에 삽입
## 결과적으로 컬럼 출력
## transaction_id, 
## quantity, 
## item_price, 
## price
## 4개의 컬럼을 출력
sql = "select * from transaction_detail_1"
sql2 = "select * from item_master"

In [22]:
sql = """
    select 
    transaction_id, 
    quantity, 
    item_price, 
    tr_d.item_id, 
    (quantity * item_price) as price 
    from 
    transaction_detail_1 as tr_d 
    left join 
    item_master as im 
    on 
    tr_d.item_id = im.item_id
"""
db.executeall(sql)

,transaction_id,quantity,item_price,item_id,price
0,T0000000113,1,210000,S005,210000
1,T0000000114,1,50000,S001,50000
2,T0000000115,1,120000,S003,120000
3,T0000000116,1,210000,S005,210000
4,T0000000117,2,85000,S002,170000
...,...,...,...,...,...
4995,T0000004865,1,120000,S003,120000
4996,T0000004866,3,50000,S001,150000
4997,T0000004867,3,50000,S001,150000
4998,T0000004868,1,210000,S005,210000


In [24]:
sql = """
        select 
        payment_date, 
        DATE_FORMAT(payment_date, '%%Y%%m') as payment_month 
        from 
        transaction_1
"""
db.executeall(sql)

,payment_date,payment_month
0,2019-02-01 01:36:57,201902
1,2019-02-01 01:37:23,201902
2,2019-02-01 02:34:19,201902
3,2019-02-01 02:47:23,201902
4,2019-02-01 04:33:46,201902
...,...,...
4995,2019-06-15 02:42:41,201906
4996,2019-06-15 03:36:16,201906
4997,2019-06-15 03:44:06,201906
4998,2019-06-15 04:14:06,201906
